# Evaluate the Performance of Machine Learning Algorithms with Resampling



- The best way to evaluate the performance of an algorithm would be to make predictions for new data to which you already know the answers. 

- The second best way is to use clever techniques from statistics called resampling methods that allow you to make accurate estimates for how well your algorithm will perform on new data.


    1. Train and Test Sets.
    2. k-fold Cross Validation.
    3. Leave One Out Cross Validation.
    4. Repeated Random Test-Train Splits.

## 1. Read data from file

In [118]:
# Evaluate using a train and a test set
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

filename = 'data/05/pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)

array = dataframe.values

X = array[:,0:8]
Y = array[:,8]

##  2. Train and Test Sets.

- The simplest method that we can use to evaluate the performance of a machine learning algorithm is to use different training and testing datasets. 

- We can take our original dataset and split it into two parts. 
    - Train the algorithm on the first part, make predictions,
    - on the second part and evaluate the predictions against the expected results.

- training set : a subset to train a model.
- test set : a subset to test the trained model.

![dataset](data/05/dataset.png)

###  2.1 Training and testing datasets using : train_test_split()

- The size of the split can depend on the size and specifics of your dataset.
- It is common to use 67% of the data for training and the remaining 33% for testing.

In [112]:
test_size = 0.33
seed = 7

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed)

### 2.2 Compute the estimated accuracy for the model 

In [107]:
model = LogisticRegression(max_iter=200)
model.fit(X_train, Y_train)

result = model.score(X_test, Y_test)

- We can see that the estimated accuracy for the model was approximately 79%.

In [113]:
print("Accuracy:",result*100.0)

Accuracy: 78.74015748031496


## 3. K-fold [Cross Validation ](https://scikit-learn.org/stable/modules/cross_validation.html) :

- Cross validation is an approach that you can use to estimate the performance of a machine learning algorithm with less variance than a single train-test set split. 
- It works by splitting the dataset into k-parts (e.g. k = 5 or k = 10). 
    - Each split of the data is called a fold. 
    - The algorithm is trained on k − 1 folds with one held back and tested on the held back fold.
    
- The result is a more reliable estimate of the performance of the algorithm on new data. 
- It is more accurate because the algorithm is trained and evaluated multiple times on different data.

### 3.1 Call KFold()

- The choice of k must allow the size of each test partition to be large enough to be a reasonable sample of the problem.

- For modest sized datasets in the thousands or tens of thousands of records, k values of 3, 5 and 10 are common. 
- In the example below we use 10-fold cross validation.

In [64]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=False)

###  3.2 Compute the mean and the standard deviation of the performance measure

In [109]:
model = LogisticRegression(max_iter=200)
results = cross_val_score(model, X, Y, cv=kfold)

- You can see that we report both the mean and the standard deviation of the performance measure. 
- When summarizing performance measures, it is a good practice to summarize the distribution of the measures, in this case assuming a Gaussian distribution of performance (a very reasonable assumption) and recording the mean and standard deviation.

In [110]:
print("Accuracy: ", (results.mean()*100.0, results.std()*100.0))

Accuracy:  (76.53543307086613, 2.235444026232818)


## 4. Leave One Out Cross Validation

- You can configure cross validation so that the size of the fold is 1 (k is set to the number of observations in your dataset). 
- This variation of cross validation is called leave-one-out cross validation. 
- The result is a large number of performance measures that can be summarized in an effort to give a more reasonable estimate of the accuracy of your model on unseen data.

- LOOCV is an extreme version of k-fold cross-validation that has the maximum computational cost. It requires one model to be created and evaluated for each example in the training dataset.
    - Don’t Use LOOCV: Large datasets or costly models to fit.
    - Use LOOCV: Small datasets or when estimated model performance is critical.

### 4.1 Call LeaveOneOut()

In [115]:
from sklearn.model_selection import LeaveOneOut

loocv = LeaveOneOut()

###  4.2 Compute the mean and the standard deviation of the performance measure

In [ ]:
model = LogisticRegression(max_iter=250)

results = cross_val_score(model, X, Y, cv=loocv)

- You can see in the standard deviation that the score has more variance than the k-fold cross validation results described above.

In [119]:
print("Accuracy: ",(results.mean()*100.0, results.std()*100.0))

Accuracy:  (77.99479166666666, 41.428150327282665)


## 5. Repeated Random Test-Train Splits

- Another variation on k-fold cross validation is to create a random split of the data like the train/test split described above, but repeat the process of splitting and evaluation of the algorithm multiple times, like cross validation.
- This has the speed of using a train/test split and the reduction in variance in the estimated performance of k-fold cross validation. 
- You can also repeat the process many more times as needed to improve the accuracy.

### 5.1 Call ShuffleSplit()

In [122]:
from sklearn.model_selection import ShuffleSplit

n_splits = 10
test_size = 0.33
seed = 7

kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)

###  5.2 Compute the mean and the standard deviation of the performance measure

In [123]:
model = LogisticRegression(max_iter=200)
results = cross_val_score(model, X, Y, cv=kfold)

- We can see that in this case the distribution of the performance measure is on par with k-fold cross validation above.

In [124]:
print("Accuracy: " , (results.mean()*100.0, results.std()*100.0))

Accuracy:  (76.53543307086613, 2.235444026232818)


## 6. What Techniques to Use When

- This section lists some tips to consider what resampling technique to use in different circumstances.
    - Generally k-fold cross validation is the gold standard for evaluating the performance of a machine learning algorithm on unseen data with k set to 3, 5, or 10.
    - Using a train/test split is good for speed when using a slow algorithm and produces performance estimates with lower bias when using large datasets.
    - Techniques like leave-one-out cross validation and repeated random splits can be useful intermediates when trying to balance variance in the estimated performance, model training speed and dataset size.

- The best advice is to experiment and find a technique for your problem that is fast and produces reasonable estimates of performance that you can use to make decisions. If in doubt, use 10-fold cross validation.